# Merging DataFrames

In [1]:
import pandas as pd

## Our Dataset
- Our datasets are spread across multiple files in this section. Each file has a `restaurant_` prefix.
- The `customers.csv` file stores our restaurant's customers.
- The `foods.csv` file stores our restaurant's menu items.
- The `week_1_sales` and `week_2_sales` files store our orders.

In [2]:
# These datasets store customer data
foods = pd.read_csv("restaurant_foods.csv")
foods.head()
customers = pd.read_csv("restaurant_customers.csv")
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [3]:
# These datasets store sales(order) data
week1 = pd.read_csv("restaurant_week_1_sales.csv")
week2 = pd.read_csv("restaurant_week_2_sales.csv")

## The pd.concat Function I
- The `concat` function concatenates one **DataFrame** to the end of another.
- **The original index labels will be kept by default.** Set `ignore_index` to True to generate a new index.
- The `keys` parameter create a **MultiIndex** using the specified keys/labels.

##### Also, the concat by default works along the vertical axis or along the direction of index. <br> This can be changed using `axis` parameter 

In [4]:
len(week1)
len(week2)
pd.concat([week1,week2])
pd.concat([week1,week2],ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


##### We might add multi-index to keep track of which dataset a record originated from.<br> This is done using the `keys` parameter

In [5]:
pd.concat([week1,week2],keys=["Week 1","Week 2"])

Customer ID  Food ID
Week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
Week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

##### Once the dataframes are concatenated into a single dataframe, we can access the records using regular `loc` / `iloc` accessors

In [6]:
allweeks = pd.concat([week1,week2],keys=["Week 1","Week 2"])
allweeks.loc[[("Week 1",4)]]

,,Customer ID,Food ID
Week 1,4,155,9


## The pd.concat Function II
- Pandas will concatenate the **DataFrames** along the row/index axis.
- Pandas will include all columns that exist in either **DataFrame**. If there are no matching values, pandas will use `NaN` values.
- We can pass the `axis` parameter an argument of `"columns"` to concatenate on the column axis.

In [7]:
pd.concat([week1,week2],keys=["Week 1","Week 2"])

Customer ID  Food ID
Week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
Week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

##### Remember that `concat` by default adds one df to the end of another df. So if columns names are same then values are added in the same column. however, if column names are different, then df2 would be added to end of df1 but due to different column names ,result will not be as expected.

In [8]:
df1 = pd.DataFrame([1,2,3],columns=["A"])
df2 = pd.DataFrame([4,5,6],columns=["B"])

In [9]:
pd.concat([df1,df2])

,A,B
0,1.0,NaN
1,2.0,NaN
2,3.0,NaN
0,NaN,4.0
1,NaN,5.0
2,NaN,6.0


#### Adding one df to the right of another df
Note that if either of the df's had direct no. of rows then the difference would show up as NaN in the output

In [10]:
pd.concat([df1,df2],axis="columns")

,A,B
0,1,4
1,2,5
2,3,6


## Left Joins
- The `merge` method joins two **DataFrames** together based on shared values in a column or an index.
- A left join merges one **DataFrame** into another based on values in the first one.
- The "left" **DataFrame** is the one we invoke the `merge` method on.
- If the left **DataFrame's** value is not found in the right **DataFrame**, the row will hold `NaN` values.<br>
<img src="SQL_Joins.png" width="700" height="400"/>

##### *Keep the left df but add from right df if the common column exists*
Joining `week1` df with the `foods` df (menu) to get the names of menu items against each purchase.<br>
Using the  `FoodID` as the common column

In [14]:
foods

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49
5,6,Pasta,13.99
6,7,Steak,24.99
7,8,Salad,11.25
8,9,Donut,0.99
9,10,Drink,1.75


In [15]:
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


#### In the `merge` method , the `on` param helps specify the common column to look for as a reference when joining the two df's 

In [12]:
week1.merge(foods,how="left",on="Food ID")

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


In [13]:
foods.merge(week1,how="left",on="Food ID")

,Food ID,Food Item,Price,Customer ID
0,1,Sushi,3.99,658
1,1,Sushi,3.99,600
2,1,Sushi,3.99,155
3,1,Sushi,3.99,341
4,1,Sushi,3.99,20
...,...,...,...,...
245,10,Drink,1.75,809
246,10,Drink,1.75,584
247,10,Drink,1.75,274
248,10,Drink,1.75,151


##### NOTE that `on` parameter can only be used when the merging column names **are SAME in BOTH the dataframes** 

## The left_on and right_on Parameters
- The `left_on` and `right_on` parameters designate the column names from each **DataFrame** to use in the merge.

##### We intend to join the `week2` df to `customers` based on the **CustomerID** and **ID** columns respectively in both df's. Note that column names are  NOT similar

In [22]:
week2.head()


,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [23]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


##### For this example, `week2` is the left dataframe while `customers` is the right dataframe

In [20]:
week2.merge(customers,how="left",left_on="Customer ID",right_on="ID")

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,688,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,813,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,495,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,189,Roger,Gordon,Male,Skilith,Operator
4,267,3,267,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...,...
245,783,10,783,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,556,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,547,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,252,Douglas,Powell,Male,Jetwire,Geologist IV


##### Note that BOTH of the 'intersecting' columns have been retained

In [21]:
week2.merge(customers,how="left",left_on="Customer ID",right_on="ID").drop(labels="ID",axis="columns")

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,Roger,Gordon,Male,Skilith,Operator
4,267,3,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...
245,783,10,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,Douglas,Powell,Male,Jetwire,Geologist IV


## Inner Joins I
- Inner joins merge two tables based on *shared*/*common* values in columns.
- If only one **DataFrame** has a value, pandas will exclude it from the final results set.
- If the same ID occurs multiple times, pandas will store each possible combination of the values.
- The design of the join ensures that the results will be the same no matter what **DataFrame** the `merge` method is invoked upon.<br>
<img src="SQL_Joins.png" width="700" height="400"/>

##### The `inner` join is basically an intersection join. For eg. we want to find the common cutomers in  Week1 and Week2

In [31]:
week1.merge(week2,how="inner",on="Customer ID",suffixes=["-Week1","-Week2"])

,Customer ID,Food ID-Week1,Food ID-Week2
0,537,9,5
1,155,9,3
2,503,5,8
3,503,5,9
4,155,1,3
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [43]:
weekm = week1.merge(week2,how="inner",on="Customer ID",suffixes=["-Week1","-Week2"])
weekm[weekm["Customer ID"]==537]

,Customer ID,Food ID-Week1,Food ID-Week2
0,537,9,5


In [41]:
week1[week1["Customer ID"]==537]

,Customer ID,Food ID
0,537,9


In [42]:
week2[week2["Customer ID"]==537]

,Customer ID,Food ID
42,537,5


##### NOTE: If a record occurs for *`m`* times in `DF1` and for *`n`* times in `DF2` then final result will contain *`m x n`* occurrences.
Checking for customer ID=155 there are 2 records in week1 with this customerID and only once in week2, so it shows up a total of 2X1 times in the resultant dataframe.

In [46]:
week1[week1["Customer ID"]==155]

,Customer ID,Food ID
4,155,9
17,155,1


In [47]:
week2[week2["Customer ID"]==155]

,Customer ID,Food ID
208,155,3


In [49]:
weekm[weekm["Customer ID"]==155]

,Customer ID,Food ID-Week1,Food ID-Week2
1,155,9,3
4,155,1,3


## Inner Joins II
- We can pass multiple arguments to the `on` parameter of the `merge` method. Pandas will require matches in both columns across the **DataFrames**.
##### In this example we need to find customers common in both weeks AND who ordered the same menu item. This means we effectively intersect the two dataframes on 2 columns.

In [51]:
week1.merge(week2, how="inner", on=["Customer ID","Food ID"]).sort_values("Customer ID")

,Customer ID,Food ID
6,21,4
4,21,4
3,233,3
0,304,3
1,540,3
7,578,5
8,578,5
5,922,1
2,937,10


In [56]:
week1[(week1["Customer ID"]==304) &(week1["Food ID"]==3) ]

,Customer ID,Food ID
55,304,3


In [57]:
week2[(week2["Customer ID"]==304) &(week2["Food ID"]==3) ]

,Customer ID,Food ID
88,304,3


##### NOTE: For multi-column joins too, if a record occurs for *`m`* times in `DF1` and for *`n`* times in `DF2` then final result will contain *`m x n`* occurrences.

## Full/Outer Join
- A **full/outer** joins values that are found in either **DataFrame** or both **DataFrames**.
- Pandas does not mind if a value exists in one **DataFrame** but not the other.
- If a value does not exist in one **DataFrame**, it will have a `NaN`.
<br>
<img src="SQL_Joins.png" width="700" height="400"/>

##### For this example, we'll combine both weeks' data into a single DataFrame. If data for a customerID exists in Week1 but not Week2, it'll show `NaN`

In [60]:
week1.merge(week2, how="outer", on=["Customer ID"],suffixes=["-Week1","-Week2"]).sort_values("Customer ID")

,Customer ID,Food ID-Week1,Food ID-Week2
0,3,2.0,NaN
1,8,NaN,6.0
2,10,2.0,NaN
3,13,NaN,2.0
4,20,1.0,NaN
...,...,...,...
449,985,5.0,NaN
450,991,2.0,NaN
451,994,NaN,2.0
452,996,NaN,10.0


In [62]:
week1[(week1["Customer ID"]==3) &(week1["Food ID"]==2) ]

,Customer ID,Food ID
96,3,2


In [63]:
week2[(week2["Customer ID"]==3) &(week2["Food ID"]==2) ]

,Customer ID,Food ID


##### As seen above, CustomerID==3 exists only in Week1 , hence Week2 shows `NaN`
##### The `indicator` parameter indicates whether a value exits in only one or both of the dataframes. This additional column will help when we want a snapshot of the joined result 

In [77]:
week1.merge(week2, how="outer", on=["Customer ID"],suffixes=["-Week1","-Week2"],indicator=True).sort_values("Customer ID")

,Customer ID,Food ID-Week1,Food ID-Week2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [81]:
week_overview = week1.merge(week2, how="outer", on=["Customer ID"],suffixes=["-Week1","-Week2"],indicator=True).sort_values("Customer ID")
week_overview

,Customer ID,Food ID-Week1,Food ID-Week2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [84]:
# The overview of all values after we joined both weeks' data
week_overview["_merge"].value_counts()

_merge
right_only    197
left_only     195
both           62
Name: count, dtype: int64

In [85]:
# We can fetch records that are ONLY common to both datasets 
week_overview[week_overview["_merge"]=="both"]

,Customer ID,Food ID-Week1,Food ID-Week2,_merge
5,21,4.0,4.0,both
6,21,4.0,4.0,both
10,30,2.0,4.0,both
14,45,7.0,8.0,both
35,75,6.0,4.0,both
...,...,...,...,...
424,922,1.0,2.0,both
430,937,10.0,10.0,both
431,937,10.0,2.0,both
436,945,5.0,4.0,both


In [87]:
# Find CustomerIDs that are exclusive -- either in week1 or week2 
week_overview[week_overview["_merge"].isin(["left_only","right_only"])]

,Customer ID,Food ID-Week1,Food ID-Week2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [89]:
con_1 = week_overview["_merge"]=="left_only"
con_2 = week_overview["_merge"]=="right_only"
week_overview[con_1| con_2]

,Customer ID,Food ID-Week1,Food ID-Week2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


## Merging by Indexes with the left_index and right_index Parameters
- Use the `on` parameter if the column(s) to be matched on have the same names in both **DataFrames**.
- Use the `left_on` and `right_on` parameters if the column(s) to be matched on have different names in the two **DataFrames**.
- Use the `left_index` or `right_index` parameters (set to True) to if the values to be matched on are found in the index of a **DataFrame**.

In [95]:
foods = pd.read_csv("restaurant_foods.csv",index_col="Food ID")
customers =pd.read_csv("restaurant_customers.csv",index_col="ID")
foods
customers

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
3,Roger,Black,Male,Tagfeed,Account Executive
4,Steven,Evans,Male,Fatz,Registered Nurse
5,Judy,Morrison,Female,Demivee,Legal Assistant
...,...,...,...,...,...
996,Debra,Garcia,Female,Dazzlesphere,Structural Engineer
997,Douglas,Bishop,Male,Livepath,Developer I
998,Frank,Franklin,Male,Brainverse,Nurse Practicioner


##### for this example, we'll join `CustomerID` column of Week1 dataframe and the **index** of the Customers dataset 
**So its left-side's <u>column</u> AND right-side's <u>index</u>**

In [97]:
week1.merge(customers,how="left",left_on="Customer ID",right_index=True)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer


##### Chaining another `merge` method to above dataframe to gring it all into a single consolidated DataFrame

In [99]:
week1.merge(customers,how="left",left_on="Customer ID",right_index=True).merge(foods,how="left",left_on="Food ID",right_index=True)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III,Burrito,9.99
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III,Donut,0.99
...,...,...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer,Donut,0.99
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant,Pasta,13.99
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor,Taco,2.99
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer,Pasta,13.99


## The join Method
- The `join` method is a shortcut for concatenating two **DataFrames** when merging by index labels.<br>
##### This method is useful when we're required to join 2 DataFrames **by index**. Its basically concating two dataframes 
We'll merge Week1 & Week1Times dataframes by index -- assuming row index as the common intersection

In [102]:
week_times = pd.read_csv("restaurant_week_1_times.csv")
week_times

,Time of Day
0,14:54:59
1,20:55:17
2,01:16:22
3,16:17:26
4,19:26:11
...,...
245,04:44:14
246,07:46:21
247,20:45:08
248,01:09:06


In [101]:
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


In [105]:
# We can use the merge method from earlier
week1.merge(week_times,how="left",left_on="Customer ID",right_index=True).sort_values("Time of Day")

,Customer ID,Food ID,Time of Day
4,155,9,00:40:33
17,155,1,00:40:33
185,167,9,00:57:48
87,117,5,01:46:15
102,64,10,01:52:35
...,...,...,...
244,621,9,NaN
245,413,9,NaN
246,926,6,NaN
248,396,6,NaN


In [106]:
# But the join method offers more simplified approach
week1.join(week_times)

,Customer ID,Food ID,Time of Day
0,537,9,14:54:59
1,97,4,20:55:17
2,658,1,01:16:22
3,202,2,16:17:26
4,155,9,19:26:11
...,...,...,...
245,413,9,04:44:14
246,926,6,07:46:21
247,134,3,20:45:08
248,396,6,01:09:06


***End of this section***
___